In [3]:
import os
import sys
import tensorflow as tf
import keras.losses
from keras import constraints, initializers, regularizers
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam, SGD, RMSprop, Adagrad, Adadelta, Adamax, Nadam
from keras.regularizers import l2
from keras.layers import Wrapper
from keras.layers import BatchNormalization, TimeDistributed, Bidirectional
from keras.layers import LeakyReLU, PReLU, ThresholdedReLU, ELU
from keras.layers import (
    Convolution2D,
    MaxPooling2D,
    Convolution1D,
    Conv1D,
    SimpleRNN,
    GRU,
    LSTM,
    CuDNNLSTM,
    CuDNNGRU,
    Conv2D,
)
from keras.layers import (
    Input,
    Lambda,
    Dense,
    Dropout,
    Flatten,
    Embedding,
    Activation,
    GRUCell,
    LSTMCell,
    SimpleRNNCell,
)
from keras.models import Model, Sequential, load_model
from keras.constraints import max_norm
from keras import regularizers, callbacks
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects
import keras
from scipy.fftpack import fft
import scipy.io.wavfile as wav
from python_speech_features import mfcc
from scipy.fftpack import dct
from scipy import signal
import json
import soundfile
from numpy.lib.stride_tricks import as_strided
from random import sample
import random
import pickle
import sys
import os
import pandas as pd
import numpy as np
# from prep import prep

# prep = prep()
# Audio processing
# Neural Network

# Setting Random Seeds
np.random.seed(95)
RNG_SEED = 95
import pandas as pd
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import wavfile #for audio processing
import os
import pickle
import pandas as pd
from collections import Counter
import librosa
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import * 
from tensorflow.keras.callbacks import Callback, ModelCheckpoint
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras import backend as K
!pip install jiwer==2.3.0
from jiwer import wer
import random
!pip install mlflow==1.26.1
import mlflow
import warnings
warnings.filterwarnings('ignore')

In [7]:

sys.path.insert(0, '../scripts/')
sys.path.append(os.path.abspath(os.path.join('..')))
import helper
from ctc_loss import CTC_loss

In [8]:
frame_step = 256
ctc = CTC_loss(frame_step)

# preprocessing model

In [6]:
from logspectrorgam import LogMelSpectrogram
def preprocessin_model(sample_rate, fft_size, frame_step, n_mels, mfcc=False):

    input_data = Input(name='input', shape=(None,), dtype="float32")
    featLayer = LogMelSpectrogram(
        fft_size=fft_size,
        hop_size=frame_step,
        n_mels=n_mels,
        
        sample_rate=sample_rate,
        f_min=0.0,
        
        f_max=int(sample_rate / 2),
    )(input_data)
    
    x = BatchNormalization(axis=2)(featLayer)
    model = Model(inputs=input_data, outputs=x, name="preprocessin_model")

    return model

# Loading our data

In [9]:
translation_obj = helper.read_obj("../data/translation_dict.pkl")
audio_obj = helper.read_obj("../data/audio_dict.pkl")
meta_data = helper.read_csv("../data/meta_data.csv")

file read as csv


In [10]:
sorted_metadata = meta_data.sort_values(by="duration")
labels = sorted_metadata['label'].to_list()
sorted_metadata

,translation,label,channel,sample_rate,duration
4133,እኛ ም እ ኮ ፉትቦል ን እንወዳ ለ ን,tr_3933_tr40034,1,8000,2.048
1476,አሸናፊ ፈጣን ተጨዋች ነው,tr_1541_tr16042,1,8000,2.048
408,እንስራ ው ተ ሸነቆረ,tr_10369_tr100091,1,8000,2.176
73,ጥያቄ ያችን ጆንያ ሙሉ ነው,tr_10067_tr098029,1,8000,2.176
3574,ኳስ ጨዋታ ኳስ ነው,tr_342_tr04042,1,8000,2.176
...,...,...,...,...,...
2614,በ አውሮፕላኗ ተመትቶ መውደቅ ሳቢያ ዜጐቻቸው ን ላ ጡት የ ብሪታኒያ ና ...,tr_2566_tr26067,1,8000,20.352
1652,ሁሴን አይዲድ እንደ ገለጹት ኢትዮጵያ ሁኔታዎች ከ ተመቻቹ ላት ሶማሊያ ን...,tr_16_tr01016,1,8000,20.736
2222,ከ ግዛታቸው ዋ ና ከተማ ጋሪ ስ ሆነው በ ስልክ ሚስተር ሞሪስ ከ ስደተኞ...,tr_2212_tr23013,1,8000,20.992
2608,የተ ለቀቁት ምርኮኞች በ አካባቢያቸው ሰላማዊ ኑሮ እንዲ ኖሩ የ ትራንስፖ...,tr_2560_tr26061,1,8000,21.120


In [ ]:
audios = []
for label in labels:
    audios.append(audio_obj[label][0])
    
translations = []
for label in labels:
    translations.append(translation_obj[label])

# Extracting audio lables

In [11]:
audios = []
for label in labels:
    audios.append(audio_obj[label][0])
    
translations = []
for label in labels:
    translations.append(translation_obj[label])

# Pridiction functions

In [15]:
def build_model(output_dim, custom_model, preprocess_model, mfcc=False, calc=None):

    input_audios = Input(name='the_input', shape=(None,))
    pre = preprocess_model(input_audios)
    pre = tf.squeeze(pre, [3])

    y_pred = custom_model(pre)
    model = Model(inputs=input_audios, outputs=y_pred, name="model_builder")
    model.output_length = calc

    return model

In [38]:
def build_model2(output_dim, cnn_model, custom_model, preprocess_model, mfcc=False, calc=None):

    input_audios = Input(name='the_input', shape=(None,))
    pre = preprocess_model(input_audios)
    pre = tf.squeeze(pre, [3])

    cnn_output = cnn_model(pre)

    y_pred = custom_model(cnn_output)
    model = Model(inputs=input_audios, outputs=y_pred, name="model_builder")
    model.output_length = calc

    return model

In [16]:
def predict(model, audio, tokenizer, int_to_char, actual=None):
    
    pred_audios = tf.convert_to_tensor([audio])
    
    y_pred = model.predict(pred_audios)

    input_shape = tf.keras.backend.shape(y_pred)
    input_length = tf.ones(shape=input_shape[0]) * tf.keras.backend.cast(input_shape[1], 'float32')
    prediction = tf.keras.backend.ctc_decode(y_pred, input_length, greedy=False)[0][0]
        
    pred = K.eval(prediction).flatten().tolist()
    pred = [i for i in pred if i != -1]
    
    predicted_text = tokenizer.decode_text(pred, int_to_char)
    
    error = None
    if actual != None:
        error = wer(actual, predicted_text)
   
    return predicted_text, error

In [18]:
from tokenizer import Tokenizer
tokenizer = Tokenizer(translations)
int_to_char, char_to_int = tokenizer.build_dict()
sample = translations[0]
encoded = tokenizer.encode(sample, char_to_int)
decoded = tokenizer.decode_text(encoded, int_to_char)

print(f"sample snt: {sample}")
print(f"encoded snt: {encoded}")
print(f"decoed snt: {decoded}")

sample snt: እኛ ም እ ኮ ፉትቦል ን እንወዳ ለ ን
encoded snt: [13, 74, 1, 16, 1, 13, 1, 67, 1, 142, 3, 122, 12, 1, 2, 1, 13, 2, 37, 39, 1, 11, 1, 2]
decoed snt: እኛ ም እ ኮ ፉትቦል ን እንወዳ ለ ን


In [19]:
helper.write_obj("../int_to_char.pkl", int_to_char)
helper.write_obj("../char_to_int.pkl", char_to_int)

# int to character or character to int

In [20]:

sample_rate = 8000
fft_size = 512
frame_step = 256
n_mels = 128

batch_size = 100
epochs = 20
data_len = len(translations)
output_dim = len(char_to_int) + 2

In [22]:
from data_generator import DataGenerator
dg = DataGenerator(translations, audios, batch_size, shuffle=True)
preprocess_model = preprocessin_model(sample_rate, fft_size, frame_step, n_mels)
preprocess_model.summary()

2022-06-09 12:39:56.188468: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-06-09 12:39:56.188516: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-172-31-60-46.ec2.internal): /proc/driver/nvidia/version does not exist
2022-06-09 12:39:56.188929: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "preprocessin_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, None)]            0         
                                                                 
 log_mel_spectrogram (LogMel  (None, None, 128, 1)     0         
 Spectrogram)                                                    
                                                                 
 batch_normalization (BatchN  (None, None, 128, 1)     512       
 ormalization)                                                   
                                                                 
Total params: 512
Trainable params: 256
Non-trainable params: 256
_________________________________________________________________


# RNN

In [24]:
from model2 import simple_rnn_model, CNN_net, BidirectionalRNN2, cnn_rnn_model

In [25]:
speech_simple_rnn = simple_rnn_model(n_mels, output_dim)
speech_simple_rnn.summary()

Model: "simple_rnn_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 the_input (InputLayer)      [(None, None, 128)]       0         
                                                                 
 rnn (GRU)                   (None, None, 223)         236157    
                                                                 
 softmax (Activation)        (None, None, 223)         0         
                                                                 
Total params: 236,157
Trainable params: 236,157
Non-trainable params: 0
_________________________________________________________________


In [26]:
simple_rnn_speech_model = build_model(output_dim, speech_simple_rnn, preprocess_model)

In [27]:
def train(model_builder, 
          data_gen,
          batch_size = 32,
          epochs=20, 
          verbose=1,
          save_path="../models/model.h5",
          optimizer=SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True, clipnorm=5),
          ):    
              
    model = ctc.add_ctc_loss(model_builder)

    checkpointer = ModelCheckpoint(filepath=save_path, verbose=0)
    model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=optimizer)
    print(model.summary())


    hist = model.fit_generator(generator=data_gen,
                               callbacks=[checkpointer],

                               epochs=epochs,
                               verbose=verbose, 
                               use_multiprocessing=False)
    return model


In [28]:
# mlflow.set_experiment('Speech Model-RNN-baseline')
# mlflow.tensorflow.autolog()
train(simple_rnn_speech_model, dg, epochs=20, save_path="../model/simple_rnn_model.h5",  batch_size=batch_size)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 the_input (InputLayer)         [(None, None)]       0           []                               
                                                                                                  
 preprocessin_model (Functional  (None, None, 128, 1  512        ['the_input[0][0]']              
 )                              )                                                                 
                                                                                                  
 tf.compat.v1.squeeze (TFOpLamb  (None, None, 128)   0           ['preprocessin_model[0][0]']     
 da)                                                                                              
                                                                                              

# Prediction

In [29]:
simple_rnn_speech_model.load_weights("../model/simple_rnn_model.h5")


actual_translation = translations[10]
sample_test_audio = audios[0]
predicted, error = predict(simple_rnn_speech_model, sample_test_audio , tokenizer, int_to_char, actual=actual_translation)

print("actual", actual_translation)
print("predicted", predicted)
print("WER: ", error)

1/1 [==============================] - 0s 368ms/step


2022-06-09 12:58:05.678197: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:690] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" vendor: "AuthenticAMD" model: "241" frequency: 2800 num_cores: 8 environment { key: "cpu_instruction_set" value: "AVX SSE, SSE2, SSE3, SSSE3, SSE4.1, SSE4.2" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 32768 l2_cache_size: 524288 l3_cache_size: 16777216 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


actual በ ባህል በ ቋንቋ አንድ ናቸው
predicted ተአንየንየንየንአንተንተንየንየንተንተንተንየ
WER:  1.0


# CNN - BiDirectional RNN

In [30]:
speech_cnn_rnn = cnn_rnn_model(n_mels, 250, 4, 1, 'same', 400, output_dim)
speech_cnn_rnn.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 the_input (InputLayer)      [(None, None, 128)]       0         
                                                                 
 conv1d (Conv1D)             (None, None, 250)         128250    
                                                                 
 bn_conv_1d (BatchNormalizat  (None, None, 250)        1000      
 ion)                                                            
                                                                 
 rnn (SimpleRNN)             (None, None, 400)         260400    
                                                                 
 batch_normalization_1 (Batc  (None, None, 400)        1600      
 hNormalization)                                                 
                                                                 
 time_distributed (TimeDistr  (None, None, 223)        8942

In [31]:
speech_cnn_rnn_model = build_model(output_dim, speech_cnn_rnn, preprocess_model)

In [32]:
train(speech_cnn_rnn_model, dg, epochs=20, save_path="../model/cnn_rnn_model.h5",  batch_size=batch_size)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 the_input (InputLayer)         [(None, None)]       0           []                               
                                                                                                  
 preprocessin_model (Functional  (None, None, 128, 1  512        ['the_input[0][0]']              
 )                              )                                                                 
                                                                                                  
 tf.compat.v1.squeeze_1 (TFOpLa  (None, None, 128)   0           ['preprocessin_model[1][0]']     
 mbda)                                                                                            
                                                                                            

In [34]:
speech_cnn_rnn_model.load_weights("../model/cnn_rnn_model.h5")
for k in range(10):
    i = random.randint(0, 3000)
    actual_translation = translations[i]
    sample_test_audio = audios[i]
    predicted, error = predict(speech_cnn_rnn_model, sample_test_audio,
                               tokenizer, int_to_char, actual=actual_translation)
   
    print("actual", actual_translation)
    print("predicted", predicted)
    print(f"WER: {error:.2f}")

1/1 [==============================] - 0s 236ms/step
actual ራሱ የ ጣሊያን ወራሪ ማስረጃ ነው
predicted ራ ታያንና ማረን
WER: 1.00
1/1 [==============================] - 0s 240ms/step
actual በ ነገው እ ለት ሁለት የ አዲስ አበባ ባላንጣ ዎች ደርቢ ያቸውን ያ ከ ና ው ና ሉ
predicted ናክ ልታለያታላንታጋ ንካና
WER: 1.00
1/1 [==============================] - 0s 24ms/step
actual የተባረሩ ኤርትራውያን ድርጅታቸው ተ ሸጦ ወኪሎ ቻቸው ከ አገር እንዲ ወጡ ተወሰነ
predicted ተባሩ ኤርታራክተክታ ለው ከገር ወ
WER: 1.00
1/1 [==============================] - 0s 24ms/step
actual የ ግብጽ ና የ ሱዱን ሚኒስትሮች ካይሮ መሰባሰባቸው ተገለጠ
predicted  ሽዳካ ን
WER: 1.00
1/1 [==============================] - 0s 23ms/step
actual ዋንኛ አላማ ዬ ለ ኢትዮጵያ ሰላም ማስገኘት ነው
predicted ንል ተፕጵያ ሻላክ
WER: 1.00
1/1 [==============================] - 0s 23ms/step
actual እሺ እኔ ትግሬ ነኝ ኢትዮጵያዊ ም ነኝ
predicted  ክነ ዮያ ነ
WER: 1.00
1/1 [==============================] - 0s 25ms/step
actual እንደ ው ስለ ተጠየቅ ኩበት ሀሳቡ ን እንድታ ገኙ ት ነው
predicted ን ለተታ ሳንታገነል
WER: 0.91
1/1 [==============================] - 0s 25ms/step
actual የ አንድ ነጥብ አራት ሚሊዮን እድሜ ያለው ቅሬተ አካል ተ

# Combining the above two models for better results

In [35]:
batch_size = 32
dg = DataGenerator(translations, audios, batch_size, shuffle=True)

In [36]:
cnn_model, cnn_shape = CNN_net(n_mels)

In [37]:
BI_RNN_2 = BidirectionalRNN2(1024, batch_size=batch_size, output_dim=output_dim)
BI_RNN_2.summary()

Model: "BidirectionalRNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 the_input (InputLayer)      [(None, None, 1024)]      0         
                                                                 
 bidirectional (Bidirectiona  (None, None, 800)        4560000   
 l)                                                              
                                                                 
 batch_normalization_5 (Batc  (None, None, 800)        3200      
 hNormalization)                                                 
                                                                 
 dropout (Dropout)           (None, None, 800)         0         
                                                                 
 bidirectional_1 (Bidirectio  (None, None, 800)        3843200   
 nal)                                                            
                                                  

In [39]:
cnn_bi_rnn_model = build_model2(output_dim, cnn_model, BI_RNN_2, preprocess_model)

In [ ]:
train(cnn_bi_rnn_model, dg, epochs=20, save_path="../models/cnn_bi_rnn_model.h5",  batch_size=batch_size)

In [45]:
cnn_bi_rnn_model.load_weights("../models/cnn_bi_rnn_model.h5")
for k in range(10):
    i = random.randint(0, 3000)
    actual_translation = translations[i]
    sample_test_audio = audios[i]
    predicted, error = predict(cnn_bi_rnn_model, sample_test_audio,
                               tokenizer, int_to_char, actual=actual_translation)
    print("actual", actual_translation)
    print("predicted", predicted)
    print(f"WER: {error:.2f}")
    print()

1/1 [==============================] - 2s 2s/step
actual እነዚህ በ ሽሬ አውራጃ የሚገኙ ወረዳ ዎች ናቸው
predicted እነዚህ በ ሽሬ የ አውራጄ የሚገኙ ወረዳዎች ናቸው
WER: 0.50

1/1 [==============================] - 2s 2s/step
actual መሪዎቹ ም ታሰሩ ተ ጋዙ ተገደሉ
predicted ህመድረች ምንካርሰሩ አጋዙ አጋ ደሉሩ
WER: 1.00

1/1 [==============================] - 0s 222ms/step
actual አጀንዳ ችን ያ ቢሆን እዚህ ቦታ ላይ ባል ተገናኘ ን ነበር
predicted አጀንዳችን ያቢ ሆን እዚህ ጐታ ላይ ባልተገናኝ ን ነዳር
WER: 0.73

1/1 [==============================] - 0s 178ms/step
actual መሬት በ መንግስት መያዙ ለ ኢኮኖሚ እንቅፋት ሆኗል ተ ባለ
predicted መሬት በ መንግስት መይያዙ ለ ኢኮለሚንክፋት ሆላል ተባለን
WER: 0.60

1/1 [==============================] - 0s 100ms/step
actual አቡነ ጳውሎስ ሜዳሊያ ተሸለሙ
predicted አቡና አውሎስት ሄዳለያ ተሸልለሙም
WER: 1.00

1/1 [==============================] - 0s 146ms/step
actual በ ዋሽንግተን ዲሲ አዲስ ራዲዮ ጣቢያ ተከፈተ
predicted በ ዋሽንግተንእንጂሺ አዲስ ራ አዲወጣ አቢያ ተከፈተ
WER: 0.71

1/1 [==============================] - 0s 170ms/step
actual የ ከተማዋ ዋ ና ዋ ና መንገዶች አስቸኳይ ጥገና እየ ተደረገላቸው ነው
predicted የከታ መዋ ዋና ዋንና መንግሮች አስቸው ኰጠ ከና እንዲ ገደረታር ናቸው 